In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import binom
np.random.seed(40)
n = 100 #total cptys
S = 125 #no. of days
m = 80 #N(0,1) cptys
d = 0.2 #additional deviation
N = 1000 #no. of simulations

In [5]:
#Return data for 1st m counterparties
returns1 = np.random.normal(loc = 0, scale = 1, size = (m, S, N))
column_names1 = [f'Stock_{i}_simu{j}' for i in range (1,m+1) for j in range(1,N+1)]
stock_data1 = pd.DataFrame(data = returns1.reshape(S,m*N),columns = column_names1)

#Return data for lst n-m counterparties
returns2 = np.random.normal(loc = 0, scale = 1+d, size = (n-m, S, N))
column_names2 = [f'Stock_{i}_simu{j}' for i in range (m+1,n+1) for j in range(1,N+1)]
stock_data2 = pd.DataFrame(data = returns2.reshape(S,(n-m)*N),columns = column_names2)

#Combine the two data
stock_data = pd.concat([stock_data1,stock_data2], axis = 1, join = 'outer')
stock_data.head()

,Stock_1_simu1,Stock_1_simu2,Stock_1_simu3,Stock_1_simu4,Stock_1_simu5,Stock_1_simu6,Stock_1_simu7,Stock_1_simu8,Stock_1_simu9,Stock_1_simu10,...,Stock_100_simu991,Stock_100_simu992,Stock_100_simu993,Stock_100_simu994,Stock_100_simu995,Stock_100_simu996,Stock_100_simu997,Stock_100_simu998,Stock_100_simu999,Stock_100_simu1000
0,-0.319326,0.702724,-0.448187,-1.924016,1.773600,-0.020066,0.930610,-1.639385,-2.203128,-0.220865,...,-0.790271,-2.498409,0.240844,-1.030518,-2.463833,-1.091837,0.800704,-0.460014,0.140474,-0.326444
1,-0.668066,0.619984,-0.692251,0.762353,-1.351146,-0.049078,0.075506,0.390483,0.536049,0.300813,...,-0.849566,0.890253,2.040691,0.460240,-1.040721,0.903035,-1.039295,1.222575,-1.262038,-1.106333
2,-1.900131,-1.429284,0.092992,-0.535411,-0.157310,-0.503677,-0.407101,-0.874252,0.806900,0.817525,...,2.044954,-0.008077,-2.389834,0.789481,0.616737,-0.168871,-1.326050,-0.933000,0.871765,0.105350
3,-1.340460,0.274883,-0.043214,-1.336269,0.092474,1.170196,-1.689815,-1.326037,-0.732246,0.529403,...,1.600528,1.258606,0.699117,1.353448,0.475318,0.466394,-0.925519,-0.524256,-0.943116,-0.262704
4,1.152396,-0.259937,1.252104,-0.504623,-2.409039,-0.935076,1.713637,-1.133347,1.162847,0.892562,...,-1.416972,0.382783,2.224912,-0.720305,0.435773,-3.719616,3.851356,1.309486,-1.938790,1.455146


In [7]:
#Qntiles to be checked
qntile = (0.9, 0.95, 0.975, 0.99)

# Counting exceptions for each stock on each simulation out of 125 days
exception_count=pd.DataFrame()
for i in range(len(qntile)):
    exception_count[i] = ((stock_data>norm.ppf(1-(1-qntile[i])/2)) | (stock_data<norm.ppf((1-qntile[i])/2))).sum()
    
exception_count.columns = qntile
exception_count

,0.900,0.950,0.975,0.990
Stock_1_simu1,14,5,2,0
Stock_1_simu2,9,7,2,2
Stock_1_simu3,9,4,4,1
Stock_1_simu4,9,3,1,1
Stock_1_simu5,13,6,3,1
...,...,...,...,...
Stock_100_simu996,18,13,10,4
Stock_100_simu997,26,17,13,8
Stock_100_simu998,22,15,9,3
Stock_100_simu999,24,13,8,4


In [9]:
# Cpty exceedance rate
stock = []
for i in range(1, n+1):
    stock.append(f'stock_{i}')

Stock_index = [element for element in stock for _ in range(N)]
exception_count['Stock'] = Stock_index
exceedance_rate_cpty = exception_count.groupby('Stock', sort = False).sum()/(N*S)
exceedance_rate_cpty

,0.9,0.95,0.975,0.99
Stock,,,,
stock_1,0.101136,0.050728,0.025712,0.010208
stock_2,0.101744,0.050336,0.025384,0.010224
stock_3,0.101592,0.050544,0.025688,0.009912
stock_4,0.099168,0.049816,0.025264,0.010240
stock_5,0.100792,0.050784,0.025512,0.010552
...,...,...,...,...
stock_96,0.169992,0.102536,0.061608,0.031824
stock_97,0.170456,0.103000,0.062112,0.031880
stock_98,0.170400,0.103344,0.062352,0.031760


In [10]:
# qntile exceedance
qntile_exception_rate = pd.DataFrame(exception_count.drop(['Stock'], axis = 1).sum()/(N*S*n))
qntile_exception_rate.columns = ['Exceedance Rate']
qntile_exception_rate

,Exceedance Rate
0.9,0.114135
0.95,0.060555
0.975,0.032443
0.99,0.014426


In [37]:
from scipy.stats import binom
qntile = (0.9, 0.95, 0.975, 0.99)
rate = [1 - x for x in qntile]
crit = pd.DataFrame(binom.ppf(0.95, n=125, p = rate))

# Calculation of K value
dummy_df = exceedance_rate_cpty.apply(lambda x: 1-x)
k_matrix = pd.DataFrame(columns = qntile)
for i in range(len(exceedance_rate_cpty)):
    k_matrix.loc[i] = norm.ppf(qntile)/norm.ppf(dummy_df.iloc[i])
for i in range(len(qntile)):
    k_matrix.iloc[:,i] = k_matrix.iloc[:,i].apply(lambda x: 1 if x<1 else x)

k_matrix.describe()

In [43]:
k_matrix.iloc[80:100].describe()

,0.900,0.950,0.975,0.990
count,20.000000,20.000000,20.000000,20.000000
mean,1.344619,1.297242,1.273359,1.255518
std,0.006161,0.004789,0.004774,0.003719
min,1.334996,1.285815,1.264365,1.248109
25%,1.340968,1.294471,1.269962,1.253640
50%,1.344894,1.297805,1.272784,1.254303
75%,1.348118,1.300339,1.275278,1.257289
max,1.359329,1.308044,1.284266,1.262857


In [23]:
# Exceedance rate simulation wise
Simu = []
for i in range(1, N+1):
    Simu.append(f'Simu_{i}')
Simu_index = Simu*n
exception_count['Simu'] = Simu_index
exceedance_rate_simu = exception_count.groupby('Simu', sort = False).sum()/(n*S)
exceedance_rate_simu

,0.9,0.95,0.975,0.99
Simu,,,,
Simu_1,0.11552,0.06184,0.03472,0.01520
Simu_2,0.11664,0.06296,0.03384,0.01576
Simu_3,0.11712,0.06240,0.03432,0.01624
Simu_4,0.11520,0.06144,0.03384,0.01496
Simu_5,0.11208,0.05984,0.03024,0.01360
...,...,...,...,...
Simu_996,0.11024,0.05720,0.03064,0.01296
Simu_997,0.11432,0.06216,0.03304,0.01544
Simu_998,0.11904,0.06160,0.03384,0.01520


In [27]:
dummy_df = exceedance_rate_simu.apply(lambda x: 1-x)
k_matrix_simu = pd.DataFrame(columns = qntile)
for i in range(len(exceedance_rate_simu)):
    k_matrix_simu.loc[i] = norm.ppf(qntile)/norm.ppf(dummy_df.iloc[i])
for i in range(len(qntile)):
    k_matrix_simu.iloc[:,i] = k_matrix_simu.iloc[:,i].apply(lambda x: 1 if x<1 else x)
exceedance_rate_simu

,0.9,0.95,0.975,0.99
Simu,,,,
Simu_1,0.11552,0.06184,0.03472,0.01520
Simu_2,0.11664,0.06296,0.03384,0.01576
Simu_3,0.11712,0.06240,0.03432,0.01624
Simu_4,0.11520,0.06144,0.03384,0.01496
Simu_5,0.11208,0.05984,0.03024,0.01360
...,...,...,...,...
Simu_996,0.11024,0.05720,0.03064,0.01296
Simu_997,0.11432,0.06216,0.03304,0.01544
Simu_998,0.11904,0.06160,0.03384,0.01520


In [49]:
### Calculation of K for overconfident counterparties

#P-value calculations
P_values = pd.DataFrame(columns = qntile)
for i in range(len(qntile)):
    P_values.iloc[:,i] = exception_count.iloc[:,i].apply(lambda x: 1- binom.cdf(x-1, S, p = 1-qntile[i]))
P_values

,0.900,0.950,0.975,0.990
Stock_1_simu1,0.369116,0.754085,0.822435,1.000000
Stock_1_simu2,0.887846,0.434789,0.822435,0.355813
Stock_1_simu3,0.887846,0.876215,0.381094,0.715292
Stock_1_simu4,0.887846,0.952296,0.957774,0.715292
Stock_1_simu5,0.483984,0.598531,0.607281,0.715292
...,...,...,...,...
Stock_100_simu996,0.073240,0.010006,0.001244,0.037449
Stock_100_simu997,0.000244,0.000173,0.000017,0.000042
Stock_100_simu998,0.006264,0.001508,0.004337,0.130684
Stock_100_simu999,0.001359,0.010006,0.013616,0.037449


In [50]:
#Ranking matrix
ranking_mat = P_values.rank(axis = 1)

#Q matrix
Q_mat = P_values.reset_index(drop = True)/ranking_mat.reset_index(drop = True)*len(qntile)
Q_mat.index = exception_count.index

final_q = Q_mat.min(axis = 1)
OvrConf = final_q < 0.0925
OvrConf.index = exception_count.index
OvrConf

Stock_1_simu1         False
Stock_1_simu2         False
Stock_1_simu3         False
Stock_1_simu4         False
Stock_1_simu5         False
                      ...  
Stock_100_simu996      True
Stock_100_simu997      True
Stock_100_simu998      True
Stock_100_simu999      True
Stock_100_simu1000     True
Length: 100000, dtype: bool

In [51]:
OvrConf_exception = exception_count[OvrConf.values]
exceedance_rate_Overconf =OvrConf_exception.groupby('Simu', sort = False).mean()/S
exceedance_rate_Overconf.sort_values(by = ['Simu'])

,0.9,0.95,0.975,0.99
Simu,,,,
Simu_1,0.166333,0.103333,0.065333,0.030000
Simu_10,0.173818,0.106182,0.066182,0.031636
Simu_100,0.171500,0.108500,0.068500,0.035500
Simu_1000,0.178909,0.115636,0.072000,0.039273
Simu_101,0.172235,0.104471,0.059765,0.031529
...,...,...,...,...
Simu_995,0.175158,0.104421,0.064421,0.036211
Simu_996,0.157200,0.099600,0.062400,0.029200
Simu_997,0.166095,0.105905,0.064000,0.032000


In [52]:
dummy_df = exceedance_rate_Overconf.apply(lambda x: 1-x)
k_matrix_OvrConf = pd.DataFrame(columns = qntile)
for i in range(len(exceedance_rate_Overconf)):
    k_matrix_OvrConf.loc[i] = norm.ppf(qntile)/norm.ppf(dummy_df.iloc[i])
for i in range(len(qntile)):
    k_matrix_OvrConf.iloc[:,i] = k_matrix_simu.iloc[:,i].apply(lambda x: 1 if x<1 else x)
k_matrix_OvrConf.describe()

,0.900,0.950,0.975,0.990
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.063721,1.061079,1.061613,1.064171
std,0.012778,0.011900,0.012269,0.014540
min,1.024659,1.017347,1.021581,1.019968
25%,1.054306,1.052906,1.053081,1.054391
50%,1.063795,1.060678,1.062004,1.064099
75%,1.072228,1.068882,1.069569,1.073567
max,1.103362,1.098786,1.097938,1.109408


In [53]:
len(OvrConf_exception)

20043

In [38]:
crit = [0]*len(qntile)
for i in range(len(rate)):
    crit[i] = norm.ppf(0.925, loc = N*rate[i], scale = np.sqrt(N*rate[i]*qntile[i]))
crit

[113.65659463497398, 59.92128598779196, 32.10711718002928, 14.529380031818949]